# Import Dependencies

In [1]:
import nltk # for text proprocessing & stemming & tokenization
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

import tflearn # Loading Model

import tensorflow # For Building Model
from tensorflow.python.framework import ops
ops.reset_default_graph()

import pickle # For Loading Processed Data
import numpy
import random

Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)
Scipy not supported!


# Import/Load Dataset

In [2]:
import os
import json

with open(os.path.join("dataset", "intents.json")) as file:
    data = json.load(file)
    # Print json file
    print(data['intents'])

[{'tag': 'greeting', 'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day', 'Whats up'], 'responses': ['Hello!', 'Good to see you again!', 'Hi there, how can I help?'], 'context_set': ''}, {'tag': 'goodbye', 'patterns': ['cya', 'See you later', 'Goodbye', 'I am Leaving', 'Have a Good day'], 'responses': ['Sad to see you go :(', 'Talk to you later', 'Goodbye!'], 'context_set': ''}, {'tag': 'age', 'patterns': ['how old', 'how old is tim', 'what is your age', 'how old are you', 'age?'], 'responses': ['I am 18 years old!', '18 years young!'], 'context_set': ''}, {'tag': 'name', 'patterns': ['what is your name', 'what should I call you', 'whats your name?'], 'responses': ['You can call me Tim.', "I'm Tim!", "I'm Tim aka Tech With Tim."], 'context_set': ''}, {'tag': 'shop', 'patterns': ['Id like to buy something', 'whats on the menu', 'what do you reccommend?', 'could i get something to eat'], 'responses': ['We sell chocolate chip cookies for $2!', 'Cookies are on the men

# Load the dataset that has been already processed.

In [3]:
try:
    with open("data.pickle", "rb") as f:
        words, labels, training, output = pickle.load(f)
except:
    print("Please run the build and train model notebook first :)")

# Load the model

In [4]:
# Load the Model
net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

# Building a Model
model = tflearn.DNN(net)

try:
    model.load("model.tflearn") # Load the model if model has already been created
except:
    print("No Saved Model, Please run the other notebook.")

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Restoring parameters from D:\(E)\Programming Tuts\Projects Tutorials\Python\Python Chatbot Tensorflow\model.tflearn


In [5]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
            
    return numpy.array(bag)

# Testing chatbot

In [6]:
def chat():
    print("Start talking with the bot (type quit to stop)!")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break

        results = model.predict([bag_of_words(inp, words)])
        results_index = numpy.argmax(results) # predicted label index
        tag = labels[results_index] # tag: greeting, goodbye, age .etc.

        # Loop through our dataset
        for tg in data["intents"]:
            if tg['tag'] == tag: # If tag is equal to the predicted tag
                # get the responses for that tag
                responses = tg['responses']
                
        print("Tag: {}".format(tag))
        print("Bot: {}".format(random.choice(responses))) # Print a random response

In [ ]:
# Run chat
chat()

Start talking with the bot (type quit to stop)!
You: is anyone there?
Tag: greeting
Bot: Hi there, how can I help?
You: Im hungry, I want to eat
Tag: shop
Bot: We sell chocolate chip cookies for $2!
You: when are you guys opening
Tag: hours
Bot: We are open 7am-4pm Monday-Friday!
You: how old are you?
Tag: age
Bot: I am 18 years old!
